In [3]:
import yfinance as yf
import pandas as pd
import plotly.graph_objs as go
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output

# Lista przykładowych spółek i ETF-ów
TICKERS = ['AAPL', 'MSFT', 'TSLA', 'NVDA', 'SPY', 'QQQ', 'VTI', 'ARKK', 'VOO']

# Widgety
ticker_dropdown = widgets.Dropdown(
    options=TICKERS,
    value='AAPL',
    description='Ticker:',
)

days_slider = widgets.IntSlider(
    value=30,
    min=5,
    max=180,
    step=5,
    description='Dni:',
)

chart_type = widgets.ToggleButtons(
    options=['Linia', 'Świeca'],
    description='Wykres:',
)

run_button = widgets.Button(description="📊 Analizuj")

output = widgets.Output()

# Główna funkcja analizy
def analyze_stock(change=None):
    with output:
        clear_output()
        symbol = ticker_dropdown.value
        days = days_slider.value
        interval = "1d"

        data = yf.download(symbol, period=f"{days}d", interval=interval)

        if data.empty:
            print("Brak danych.")
            return

        # Wykresy
        if chart_type.value == 'Linia':
            fig = go.Figure()
            fig.add_trace(go.Scatter(x=data.index, y=data['Close'], name='Zamknięcie'))
        else:
            fig = go.Figure(data=[go.Candlestick(
                x=data.index,
                open=data['Open'],
                high=data['High'],
                low=data['Low'],
                close=data['Close']
            )])

        fig.update_layout(title=f'{symbol} - Ostatnie {days} dni', xaxis_title='Data', yaxis_title='Cena (USD)')
        fig.show()

        # Statystyki
        pct_change = (data['Close'][-1] - data['Close'][0]) / data['Close'][0] * 100
        print(f"\n📈 Zmiana procentowa: {pct_change:.2f}%")
        print(f"📅 Od: {data.index[0].date()}  Do: {data.index[-1].date()}")
        print(f"🔻 Najniższa: {data['Close'].min():.2f} USD")
        print(f"🔺 Najwyższa: {data['Close'].max():.2f} USD")

# Połączenie
run_button.on_click(analyze_stock)

dashboard = widgets.VBox([
    widgets.HBox([ticker_dropdown, days_slider, chart_type]),
    run_button,
    output
])

display(dashboard)


Matplotlib is building the font cache; this may take a moment.
